In [1]:
import requests
import json
import pandas as pd

In [ ]:
url ="http://publicplansdata.org/api/"

In [ ]:
#Make sure to use QDataset, NOT QDataSet.  Makes a big difference
paramsratios = {
    "q": "QDataset",
    "dataset": "pensiongasbschedules",
     "format": "json",
    #"variables": 
     #"v": variables,
}
paramsbasicplandata = {
    "q": "QDataset",
    "dataset": "pensionplanbasics",
     "format": "json",
    #"variables": 
     #"v": ,
}

paramsbasicinvestperform = {
    "q": "QDataset",
    "dataset": "pensioninvestmentperformancedetailed_actl",
     "format": "json",
    #"variables": 
     #"v": ,
}

paramsinvestreturn = {
    "q": "QDataset",
    "dataset": "pensioninvestmentreturn",
     "format": "json",
    #"variables": 
     #"v": ,
}

paramsreturnassumptions = {
    "q": "QDataset",
    "dataset": "pensiongasbassumptions",
     "format": "json",
    #"variables": 
     #"v": ,
}

paramspenincomestatement = {
    "q": "QDataset",
    "dataset": "pensionincomestatement",
     "format": "json",
    #"variables": 
     #"v": ,
}

paramspenactliab = {
    "q": "QDataset",
    "dataset": "pensionactuarialliabilities",
     "format": "json",
    #"variables": 
     #"v": ,
}

In [ ]:
#create requests
ratiosresponse = requests.get(url, params = paramsratios)
plandataresponse = requests.get(url,params=paramsbasicplandata)
investperformresponse = requests.get(url,params=paramsbasicinvestperform)
investreturnresponse = requests.get(url,params=paramsinvestreturn)
returnassumptionsresponse = requests.get(url,params=paramsreturnassumptions)
penincomestatementresponse = requests.get(url,params=paramspenincomestatement)
penactliabresponse = requests.get(url,params=paramspenactliab )

In [ ]:
#create json response object
fundedratiosdata = ratiosresponse.json()
plandata = plandataresponse.json()
investperform = investperformresponse.json()
investreturn = investreturnresponse.json()
returnassumptions = returnassumptionsresponse.json()
penincomestatement=penincomestatementresponse.json()
penactliab = penactliabresponse.json()

In [ ]:
#change response object into DataFrame
funded_ratios_full = pd.DataFrame(fundedratiosdata[1:])
plan_data_full = pd.DataFrame(plandata[1:])
investperform_data_full=pd.DataFrame(investperform[1:])
investreturn_data_full=pd.DataFrame(investreturn[1:])
returnassumptions_data_full=pd.DataFrame(returnassumptions[1:])
penincomestatement_data_full=pd.DataFrame(penincomestatement[1:])
penactliab_data_full=pd.DataFrame(penactliab[1:])